# Exercise List 2

## Exercise 1

Dependencies

In [19]:
import sys
from pandas import DateOffset
sys.path.append(f'../FinanceHub')

from calendars import DayCounts
import pandas as pd

In [20]:
starting_date = '2021-02-05'

# inputs: string in format yyyy-mm-dd
def payout_dates(reference_date, expiry_date):
    dc = DayCounts('BUS/252', calendar='anbima')

    t = dc.following(pd.to_datetime(reference_date))
    T = dc.following(pd.to_datetime(expiry_date))

    d_years = 2 * (T.year - t.year) - 1
    # Check what's the first payout day
    first_day = pd.to_datetime(f"{t.year}-01-01")
    second_day = pd.to_datetime(f"{t.year}-07-01")
    if t != first_day:
        if t > second_day:
            d_years -= 1
            starting_day = pd.to_datetime(f"{t.year+1}-01-01")
        else:
            starting_day = second_day
    else:
        starting_day = first_day

    output = [dc.following(starting_day)]
    # Iterate over the years and append business day for payouts
    for i in range(0, d_years-1):
        if output[i].month == 1:
            output.append(
                dc.following(pd.to_datetime(f"{output[i].year}-07-01"))
            )
        else:
            output.append(
                dc.following(pd.to_datetime(f"{output[i].year+1}-01-01"))
            )
    # Add last payout date, expiry
    output.append(dc.following(pd.to_datetime(expiry_date)))

    return output

payout = payout_dates('2021-02-05', '2023-01-01')
payout_df = pd.DataFrame(payout, columns=["Payout Dates"])
payout_df

,Payout Dates
0,2021-07-01
1,2022-01-03
2,2022-07-01
3,2023-01-02


## Exercise 2

In [21]:
default_coupon = 0.1

def calc_pu(reference_date, expiry_date, yield_rate, coupon=default_coupon, face_value=1000):
    dc = DayCounts('BUS/252', calendar='anbima')
    # Calculate the payout dates
    payout_days = payout_dates(reference_date, expiry_date)

    # First coupon value from reference date
    t = dc.following(pd.to_datetime(reference_date))
    du = dc.days(t, payout[0])
    # Constant values: Coupon value and yield rate
    c = (1 + coupon)**0.5 - 1
    y = (1 + yield_rate)
    _pu = c/(y**(du/252))
    for i in range(1, len(payout_days)):
        # Calculate business days between each payout
        du += dc.days(payout_days[i-1], payout_days[i])
        # Sums up coupon values
        _pu += c/(y**(du/252))

    # Last payout of face value
    _pu += 1/(y**(du/252))

    return face_value*_pu

yields = [0.051132,0.062152,0.068692,0.073165,0.076390]
expiry_dates = [
    '2023-01-01',
    '2025-01-01',
    '2027-01-01',
    '2029-01-01',
    '2031-01-01'
]

prices = []
for yf, exp in zip(yields, expiry_dates):
    prices.append(calc_pu(starting_date, exp, yf))

prices_df = pd.DataFrame(prices, columns=["PU"])
prices_df

,PU
0,1094.209742
1,1134.782973
2,1154.793262
3,1164.388256
4,1167.734555


## Exercise 3

In [22]:
def calc_yield(reference_date, expiry_date, price, coupon=default_coupon, face_value=1000, estimate=0.05):
    from scipy import optimize
    get_yield = lambda yield_rate: calc_pu(reference_date, expiry_date, yield_rate) - price
    # Use the function above and an estimate of the yield rate to optimize
    # the function until we find a good enough value for the yield so that:
    # calc_pu(estimate_yield) = price
    return optimize.newton(get_yield, estimate)

prices = [ 1094.209749, 1134.782985, 1154.793279, 1164.388278, 1167.734580 ]

yield_rates = []
for pu, exp in zip(prices, expiry_dates):
    yield_rates.append(calc_yield(starting_date, exp, pu))

yields_df = pd.DataFrame(yield_rates, columns=["Yield Rate"])
yields_df

,Yield Rate
0,0.051132
1,0.062152
2,0.068692
3,0.073165
4,0.076390
